In [3]:
from basic import *

以下引入数据，`family_numbers`为各年总家庭数，`rural_numbers`为各年样本中的农村总数，

In [4]:
family_numbers = multicount('famecon')

agri_numbers = multicount(
    'famecon',
    conditions=['fk1=1'] + ['fk1l=1'] * 4)

rural_numbers = multicount(
    'famecon',
    conditions="urban{year}=0",
    condition_format=['', '12', '14', '16', '18']
    )

rural_agri_numbers = multicount(
    'famecon',
    conditions=[
        'urban=0 and fk1=1',
        'urban12=0 and fk1l=1',
        'urban14=0 and fk1l=1',
        'urban16=0 and fk1l=1',
        'urban18=0 and fk1l=1'
    ]
)

计算农村从事农业的百分比和从事农业的人口在农村的比例：

In [5]:
agri_ratios_in_rural = [format(100*b/a, '.2f') for a, b in zip(rural_numbers, rural_agri_numbers)]

rural_ratios_in_agri = [format(100*a/b, '.2f') for a, b in zip(rural_agri_numbers, agri_numbers)]

print(agri_ratios_in_rural, rural_ratios_in_agri)

['79.71', '78.63', '73.23', '73.18', '69.24'] ['78.65', '77.21', '75.67', '74.23', '74.78']


计算2018年农业花费
（从事农业的家庭在各个部分的花费）
`fl3`为是否从事种植业林业，`fl6`为是否从事牧渔业

需要稍微清洗一下数据：如果出现-1，或者5项均为0，判定为无效数据

In [33]:
plant_columns = ["fl501", "fl502", "fl503", "fl504", "fl505"]
plant_columns_zh = ["种子化肥农药费", "雇工费", "机器租赁费", "灌溉费", "其他费用"]
animal_columns = ["fl801", "fl802", "fl803", "fl804", "fl805"]
animal_columns_zh = ["种畜鱼苗费", "雇工费", "机器租赁费", "饲料费", "其他花费"]

plant_data = get('SELECT fl501, fl502, fl503, fl504, fl505 FROM famecon_2018 WHERE fl3 = 1')
animal_data = get('SELECT fl801, fl802, fl803, fl804, fl805 FROM famecon_2018 WHERE fl3 = 1')

# 清洗数据：
indexes = []
def map_condition(family):
    # return  not (family[0] == -1 or family[0] == -8 \
    # or family[1] == -1 or family[1] == -8 \
    # or family[2] == -1 or family[2] == -8 \
    # or family[3] == -1 or family[3] == -8 \
    # or family[4] == -1 or family[4] == -8 \
    # or family == (0, 0, 0, 0, 0))
    family = list(family)
    for i in range(len(family)):
        # 将-1， -8转换为0
        if family[i] is None or family[i]<0:
            family[i]=0
    return tuple(family)

def filter_cond(family):
    # 过滤全为0的数值
    return family != (0, 0, 0, 0, 0)

plant_data_washed = list(filter(filter_cond, map(map_condition, plant_data)))
animal_data_washed = list(filter(filter_cond, map(map_condition, animal_data)))

绘制箱线图，箱线图需要的是最小值、1/4分位点、中位数、3/4分位点和最大值

def get_boxplot_params(data):
    n = len(data)
    data.sort()
    # Python从0开始计数，0,1,2,...,n的中间数是(n-1)/2
    median = data[int((n-1)/2)] if n%2==1 else (data[int(n/2)] + data[int((n-1)/2)])/2
    q1 = data[int(n/4)]
    q3 = data[int(3*n/4)]
    iqr = q3 - q1
    up = q3 + 1.5 * iqr
    down = q1 - 1.5 * iqr if q1 - 1.5 * iqr > 0 else 0
    return [down, q1, median, q3, up]

def transpose(data):
    '''
    将二维数组转置
    '''
    transposed = list(map(list, zip(*data)))
    return transposed

plant_boxplot = []
for data in transpose(plant_data_washed):
    plant_boxplot.append(get_boxplot_params(data))
    
animal_boxplot = []
for data in transpose(animal_data_washed):
    animal_boxplot.append(get_boxplot_params(data))

print('The plant is: ',plant_boxplot)
print('The animal is ', animal_boxplot)

绘制条形图，各组取平均

In [45]:
def transpose(data):
    '''
    将二维数组转置
    '''
    transposed = list(map(list, zip(*data)))
    return transposed

plant_average = [int(sum(row)/len(row)) for row in transpose(plant_data_washed)]
animal_average = [int(sum(row)/len(row)) for row in transpose(animal_data_washed)]

print(plant_average, animal_average)

[4651, 924, 581, 401, 468] [2996, 245, 60, 4149, 649]


同理，计算2014年的数据

In [46]:
plant_data_2014 = get('SELECT fl501, fl502, fl503, fl504, fl505 FROM famecon_2014 WHERE fl3 = 1')
animal_data_2014 = get('SELECT fl801, fl802, fl803, fl804, fl805 FROM famecon_2014 WHERE fl3 = 1')

plant_data_2014
# 清洗数据：
plant_data_washed_2014 = list(filter(filter_cond, map(map_condition, plant_data_2014)))
animal_data_washed_2014 = list(filter(filter_cond, map(map_condition, animal_data_2014)))

plant_average14 = [int(sum(row)/len(row)) for row in transpose(plant_data_washed_2014)]
animal_average14 = [int(sum(row)/len(row)) for row in transpose(animal_data_washed_2014)]

print(plant_average14, animal_average14)

[4144, 641, 428, 298, 353] [2528, 120, 20, 5064, 351]


同理2016年：

In [37]:
plant_data16 = get('SELECT fl501, fl502, fl503, fl504, fl505 FROM famecon_2016 WHERE fl3 = 1')
animal_data16 = get('SELECT fl801, fl802, fl803, fl804, fl805 FROM famecon_2016 WHERE fl3 = 1')

plant_data16
# 清洗数据：
plant_data_washed16 = list(filter(filter_cond, map(map_condition, plant_data16)))
animal_data_washed16 = list(filter(filter_cond, map(map_condition, animal_data16)))

plant_average16 = [int(sum(row)/len(row)) for row in transpose(plant_data_washed16)]
animal_average16 = [int(sum(row)/len(row)) for row in transpose(animal_data_washed16)]

print(plant_average16, animal_average16)

[4728, 732, 592, 335, 381] [3933, 120, 24, 4958, 404]


2012年的数据稍微麻烦一些，由于机器租赁费和灌溉费没有分类，我们需要结合后3年数据将它们做分解。
同样地，畜牧业渔业直接缺失了机器租赁费，也需要做分解

In [39]:
plant_machine_cost_ratios = [pa[2]/(pa[2]+pa[3]) for pa in [plant_average14, plant_average16, plant_average]]
animal_machine_cost_ratios = [aa[2]/(aa[2]+aa[4]) for aa in [animal_average14, animal_average16, animal_average]]

plant_machine_ratio = sum(plant_machine_cost_ratios)/len(plant_machine_cost_ratios)
animal_machine_ratio = sum(animal_machine_cost_ratios)/len(animal_machine_cost_ratios)

print(plant_machine_ratio, animal_machine_ratio)

0.606467045494215 0.06558534674998019


In [44]:
plant_data12 = get('SELECT fl401, fl402, fl403, fl404 FROM famecon_2012 WHERE fl2 = 1')
animal_data12 = get('SELECT fl901, fl902, fl903, fl904 FROM famecon_2012 WHERE fl7 = 1')

# 清洗数据：
plant_data_washed12 = list(filter(filter_cond, map(map_condition, plant_data12)))
animal_data_washed12 = list(filter(filter_cond, map(map_condition, animal_data12)))

plant_average12 = [int(sum(row)/len(row)) for row in transpose(plant_data_washed12)]
animal_average12 = [int(sum(row)/len(row)) for row in transpose(animal_data_washed12)]

plant_average12_fix = [plant_average12[0], plant_average12[1], int(plant_average12[2] * plant_machine_ratio), int(plant_average12[2] * (1 - plant_machine_ratio)), plant_average12[3] ]
animal_average12_fix = [animal_average12[0], animal_average12[1], int(animal_average12[3] * animal_machine_ratio), plant_average12[2], int(animal_average12[3] * (1 - plant_machine_ratio)) ]

print(plant_average12_fix, animal_average12_fix)

[2956, 578, 214, 139, 218] [475, 0, 10, 354, 63]
